In [1]:
from spx_analysis import simulate_market 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import datetime as dt, timedelta

In [48]:
spx = pd.read_csv('^GSPC.csv', index_col=0)

spx.index = pd.to_datetime(spx.index).year
spx['daily_chg'] = np.log(spx['Adj Close']) - np.log(spx['Adj Close'].shift(1))

In [90]:

years = 60

start_date = dt(2020, 1, 1)
end_date = dt(2080, 12, 31)
days = pd.date_range(start_date, end_date, freq='B')

# Random market
market = simulate_market(start_date, end_date, spx['daily_chg'][-20*252:]*100)
market.set_index(days, inplace=True)

# Annualizing returns
market = market.asfreq('Y', method='ffill', normalize=True)
market.index = market.index.year

15915


## Combining with investment profile

In [107]:
inv_plan = pd.read_csv('investment_plan.csv', sep=';', index_col=0)
market['Price'] = market['Price']*100/market['Price'].iloc[0]

In [114]:
def calculate_return(inv, market):
    assert len(inv) == len(market), 'Investment plan should be same no of periods as market' 
    inv['Roll_contribution'] = inv['Contributions'].cumsum()
    inv['Price'] = market['Price']
    inv['Account_value'] = inv['Roll_contribution']*inv['Gearing']
    
    inv['Notional_debt'] = 0
    
    for i in range(len(inv)):
        gearing = inv.iloc[i, 0]
        cont = inv.iloc[i, 1]
        inv.iloc[i, 6] = max(gearing-1, 0)*cont

        
    inv['Total_notional_debt'] = inv['Notional_debt'].cumsum()
    inv['Total_debt'] = 0
    inv['Total_debt'].iloc[0] = inv['Total_debt'].iloc[0] 
    inv['Accrued_interest'] = 0
    
    
    for i in range(len(inv)):
        debt_cost = inv.iloc[i, 2]
        total_debt = inv.iloc[i, 1]
        #inv.iloc[i, 6] = max(gearing-1, 0)*cont

    return inv

In [115]:
test = calculate_return(inv_plan, market)
print(test)

      Gearing  Contributions  Debt cost  Roll_contribution          Price  \
Year                                                                        
2020     2.00         100000       0.03             100000     100.000000   
2021     2.00         105000       0.03             205000     111.990431   
2022     2.00         110250       0.03             315250     178.144121   
2023     2.00         115763       0.03             431013     192.417568   
2024     2.00         121551       0.03             552564     204.588567   
...       ...            ...        ...                ...            ...   
2076     0.79         532877       0.03           21870919   43338.825399   
2077     0.77         527548       0.03           22398467   47968.354162   
2078     0.76         522273       0.03           22920740   70878.500767   
2079     0.74         517050       0.03           23437790   87423.178121   
2080     0.73         511879       0.03           23949669  140438.273076   

/Users/nielseriksen/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
